날짜 변환<br/>
결측 값 행 제거 + 결측 값 많은 열 제거 + 수치 데이터 정규화 작업<br/>
10 단위 1로 수정<br/>
null data 분리<br/>
ATA 자체를 삭제<br/>

In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
train_df = pd.read_csv('../../../train.csv')
test_df = pd.read_csv('../../../test.csv')
train_df.to_parquet('../../../train.parquet')
test_df.to_parquet('../../../test.parquet')

In [3]:
train_df.head()

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444


In [4]:
test_df.head()

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE
0,TEST_000000,SG,GIW5,Container,1.826589,2020-06-18 11:58,K322654,50.0,18,117000,...,109000,340.0,KQMD74,Panama,0.37,1.63,27.1,1.587063,19,0.002615
1,TEST_000001,CN,WEY7,Cargo,25.399386,2021-05-26 22:20,E376681,10.0,13,3810,...,2560,80.0,LBYS27,Belize,-2.79,-2.33,14.2,2.663972,6,0.001028
2,TEST_000002,CN,NGG6,Container,111.079467,2019-12-16 0:09,F811111,20.0,26,10900,...,8440,140.0,EKFV15,Singapore,0.04,-4.91,9.3,3.255315,8,0.001743
3,TEST_000003,CA,FFM2,Bulk,9.175258,2015-11-16 5:30,A737561,30.0,9,55800,...,31500,190.0,MASW32,Panama,NaN,NaN,NaN,NaN,2,0.000182
4,TEST_000004,JP,QYY1,Container,0.000000,2018-10-24 1:11,A827175,30.0,19,39800,...,33000,220.0,SIEL54,Liberia,NaN,NaN,NaN,NaN,10,0.000552


In [5]:
train_df = pd.read_parquet('../../../train.parquet').drop(columns=['SAMPLE_ID'])
test_df = pd.read_parquet('../../../test.parquet').drop(columns=['SAMPLE_ID'])

In [6]:
# datetime 컬럼 제거
train_df.drop(columns=['ATA'], inplace=True)
test_df.drop(columns=['ATA'], inplace=True)

In [7]:
assert sorted(train_df["ARI_CO"].unique()) == sorted(test_df["ARI_CO"].unique())

assert sorted(train_df["ARI_PO"].unique()) == sorted(test_df["ARI_PO"].unique())

assert sorted(train_df["SHIP_TYPE_CATEGORY"].unique()) == sorted(test_df["SHIP_TYPE_CATEGORY"].unique())

In [8]:
scaler = StandardScaler()
x = train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]]
scaler.fit(x)
train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]] = scaler.transform(x)
test_feautre = test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]]
test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH"]] = scaler.transform(test_feautre)

In [9]:
# train, test 동일한 경우
original_train_columns = set(train_df.columns)
original_test_columns = set(test_df.columns)
train_df = pd.get_dummies(train_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY", "FLAG"], drop_first=True)
test_df = pd.get_dummies(test_df, columns=["ARI_CO", "ARI_PO", "SHIP_TYPE_CATEGORY", "FLAG"], drop_first=True)
new_train_columns = set(train_df.columns)
new_test_columns = set(test_df.columns)

subset_train_test = new_train_columns - new_test_columns
subset_test_train = new_test_columns - new_train_columns
print(len(new_train_columns), len(new_test_columns))
print(subset_train_test)
print(subset_test_train)

270 262
{'FLAG_Moldova', 'FLAG_Bulgaria', 'FLAG_Azerbaijan', 'FLAG_Equatorial Guinea', 'FLAG_Comoros False', 'FLAG_Sao Tome & Principe False', 'FLAG_Guyana False', 'CI_HOUR', 'FLAG_Libya'}
{'FLAG_Gambia'}


In [10]:
FLAG_train_columns = [column for column in train_df.columns if "FLAG" in column]
FLAG_test_columns = [column for column in test_df.columns if "FLAG" in column]
ARI_CO_train_columns = [column for column in train_df.columns if "ARI_CO" in column]
ARI_CO_test_columns = [column for column in test_df.columns if "ARI_CO" in column]
ARI_PO_train_columns = [column for column in train_df.columns if "ARI_PO" in column]
ARI_PO_test_columns = [column for column in test_df.columns if "ARI_PO" in column]
SHIP_TYPE_CATEGORY_train_columns = [column for column in train_df.columns if "SHIP_TYPE_CATEGORY" in column]
SHIP_TYPE_CATEGORY_test_columns = [column for column in test_df.columns if "SHIP_TYPE_CATEGORY" in column]
print(FLAG_train_columns)
print(FLAG_test_columns)
print(ARI_CO_train_columns)
print(ARI_CO_test_columns)
print(ARI_PO_train_columns)
print(ARI_PO_test_columns)
print(SHIP_TYPE_CATEGORY_train_columns)
print(SHIP_TYPE_CATEGORY_test_columns)

['FLAG_Antigua & Barbuda', 'FLAG_Argentina', 'FLAG_Australia', 'FLAG_Azerbaijan', 'FLAG_Bahamas', 'FLAG_Bahrain', 'FLAG_Bangladesh', 'FLAG_Barbados', 'FLAG_Belgium', 'FLAG_Belize', 'FLAG_Bermuda', 'FLAG_Brazil', 'FLAG_Brunei', 'FLAG_Bulgaria', 'FLAG_Cambodia', 'FLAG_Cameroon', 'FLAG_Canada', 'FLAG_Cayman Islands', 'FLAG_Chile', "FLAG_China, People's Republic Of", 'FLAG_Chinese Taipei', 'FLAG_Comoros', 'FLAG_Comoros False', 'FLAG_Cook Islands', 'FLAG_Croatia', 'FLAG_Curacao', 'FLAG_Cyprus', 'FLAG_Denmark (Dis)', 'FLAG_Djibouti', 'FLAG_Dominica', 'FLAG_Egypt', 'FLAG_Equatorial Guinea', 'FLAG_Equatorial Guinea False', 'FLAG_Estonia', 'FLAG_Ethiopia', 'FLAG_Faeroes (Fas)', 'FLAG_Finland', 'FLAG_France', 'FLAG_France (Fis)', 'FLAG_Gabon', 'FLAG_Georgia', 'FLAG_Germany', 'FLAG_Ghana', 'FLAG_Gibraltar', 'FLAG_Greece', 'FLAG_Guinea-Bissau', 'FLAG_Guyana', 'FLAG_Guyana False', 'FLAG_Honduras', 'FLAG_Hong Kong, China', 'FLAG_India', 'FLAG_Indonesia', 'FLAG_Iran', 'FLAG_Irish Republic', 'FLAG_Isl

In [11]:
test_df[list(subset_train_test)] = 0 # train에는 있고 test에는 없는 건 value를 0으로 채움.
test_df = test_df.drop(columns=list(subset_test_train)) # test에만 있는 column은 drop함.
print(len(train_df.columns), len(test_df.columns))
assert sorted(train_df.columns) == sorted(test_df.columns)

for column in FLAG_train_columns + SHIP_TYPE_CATEGORY_train_columns + ARI_PO_train_columns + ARI_CO_train_columns: # 이상하게 train의 column 내부 값이 False, True로 나와서 astype으로 int형으로 변환함.
    print(column)
    train_df[column] = train_df[column].astype(int)
    test_df[column] = test_df[column].astype(int)

270 270
FLAG_Antigua & Barbuda
FLAG_Argentina
FLAG_Australia
FLAG_Azerbaijan
FLAG_Bahamas
FLAG_Bahrain
FLAG_Bangladesh
FLAG_Barbados
FLAG_Belgium
FLAG_Belize
FLAG_Bermuda
FLAG_Brazil
FLAG_Brunei
FLAG_Bulgaria
FLAG_Cambodia
FLAG_Cameroon
FLAG_Canada
FLAG_Cayman Islands
FLAG_Chile
FLAG_China, People's Republic Of
FLAG_Chinese Taipei
FLAG_Comoros
FLAG_Comoros False
FLAG_Cook Islands
FLAG_Croatia
FLAG_Curacao
FLAG_Cyprus
FLAG_Denmark (Dis)
FLAG_Djibouti
FLAG_Dominica
FLAG_Egypt
FLAG_Equatorial Guinea
FLAG_Equatorial Guinea False
FLAG_Estonia
FLAG_Ethiopia
FLAG_Faeroes (Fas)
FLAG_Finland
FLAG_France
FLAG_France (Fis)
FLAG_Gabon
FLAG_Georgia
FLAG_Germany
FLAG_Ghana
FLAG_Gibraltar
FLAG_Greece
FLAG_Guinea-Bissau
FLAG_Guyana
FLAG_Guyana False
FLAG_Honduras
FLAG_Hong Kong, China
FLAG_India
FLAG_Indonesia
FLAG_Iran
FLAG_Irish Republic
FLAG_Isle Of Man
FLAG_Israel
FLAG_Italy
FLAG_Jamaica
FLAG_Japan
FLAG_Jordan
FLAG_Kiribati
FLAG_Korea, North
FLAG_Korea, South
FLAG_Kuwait
FLAG_Latvia
FLAG_Lebanon
F

In [12]:
train_df.head()

,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,FLAG_Togo False,FLAG_Turkey,FLAG_Tuvalu,FLAG_Ukraine,FLAG_United Arab Emirates,FLAG_United Kingdom,FLAG_United States Of America,FLAG_Unknown,FLAG_Vanuatu,FLAG_Vietnam
0,0.373515,Z618338,30.0,1.313464,-0.618393,10.0,10.0,-0.627129,-0.331094,CQSB78,...,0,0,0,0,0,0,0,0,0,0
1,-0.624901,X886125,30.0,-0.248154,-0.435760,10.0,10.0,-0.454120,-0.331094,SPNO34,...,0,0,0,0,0,0,0,0,0,0
2,-0.624901,T674582,50.0,-0.390119,1.297676,30.0,20.0,2.509917,2.173065,FNPK22,...,0,0,0,0,0,0,0,0,0,0
3,-0.624901,Y847238,20.0,0.461672,-0.892185,10.0,10.0,-0.914627,-1.121880,PBZV77,...,0,0,0,0,0,0,0,0,0,0
4,0.249255,A872328,50.0,-0.674049,0.825350,20.0,10.0,1.405718,1.250480,GUCE76,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test_df.head()

,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,FLAG_Vietnam,FLAG_Moldova,FLAG_Bulgaria,FLAG_Azerbaijan,FLAG_Equatorial Guinea,FLAG_Comoros False,FLAG_Sao Tome & Principe False,FLAG_Guyana False,CI_HOUR,FLAG_Libya
0,-0.565845,K322654,50.0,0.461672,0.841094,30.0,20.0,1.721203,1.777671,KQMD74,...,0,0,0,0,0,0,0,0,0,0
1,0.196288,E376681,10.0,-0.248154,-0.940992,10.0,10.0,-0.986884,-1.649072,LBYS27,...,0,0,0,0,0,0,0,0,0,0
2,2.966415,F811111,20.0,1.597394,-0.829365,10.0,10.0,-0.837283,-0.858285,EKFV15,...,0,0,0,0,0,0,0,0,0,0
3,-0.328255,A737561,30.0,-0.816014,-0.122451,20.0,10.0,-0.250581,-0.199296,MASW32,...,0,0,0,0,0,0,0,0,0,0
4,-0.624901,A827175,30.0,0.603638,-0.374358,20.0,10.0,-0.212418,0.196098,SIEL54,...,0,0,0,0,0,0,0,0,0,0


In [14]:
for name in ["BREADTH", "DEPTH", "DRAUGHT"]:
    train_df[name] = train_df[name].apply(lambda x: x // 10)
    test_df[name] = test_df[name].apply(lambda x: x // 10)

In [15]:
train_df = train_df.dropna(subset=["BREADTH", "DEPTH", "DRAUGHT", "LENGTH"], axis=0)

In [ ]:
# train_dummies = pd.get_dummies(train_df, columns=["FLAG"])
# test_dummies = pd.get_dummies(test_df, columns=["FLAG"])

In [38]:
# train_dummies.head()

,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,FLAG_Togo False,FLAG_Turkey,FLAG_Tuvalu,FLAG_Ukraine,FLAG_United Arab Emirates,FLAG_United Kingdom,FLAG_United States Of America,FLAG_Unknown,FLAG_Vanuatu,FLAG_Vietnam
0,0.373515,Z618338,3.0,1.313464,-0.618393,1.0,1.0,-0.627129,-0.331094,CQSB78,...,False,False,False,False,False,False,False,False,False,False
1,-0.624901,X886125,3.0,-0.248154,-0.435760,1.0,1.0,-0.454120,-0.331094,SPNO34,...,False,False,False,False,False,False,False,False,False,False
2,-0.624901,T674582,5.0,-0.390119,1.297676,3.0,2.0,2.509917,2.173065,FNPK22,...,False,False,False,False,False,False,False,False,False,False
3,-0.624901,Y847238,2.0,0.461672,-0.892185,1.0,1.0,-0.914627,-1.121880,PBZV77,...,False,False,False,False,False,False,False,False,False,False
4,0.249255,A872328,5.0,-0.674049,0.825350,2.0,1.0,1.405718,1.250480,GUCE76,...,False,False,False,False,False,False,False,False,False,False


In [21]:
# train_df, test_df = train_dummies.align(test_dummies, axis=1, fill_value=0)

In [22]:
# assert sorted(list(train_df.columns)) == sorted(list(test_df.columns))

In [16]:
categorical_features = ['ID', 'SHIPMANAGER', 'PORT_SIZE']
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  train_df[feature] = le.fit_transform(train_df[feature].astype(str))
  le_classes_set = set(le.classes_)
  test_df[feature] = test_df[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test_df[feature] = le.transform(test_df[feature].astype(str))
  encoders[feature] = le

In [17]:
not_null_idx = train_df[train_df["U_WIND"].notna() & train_df["V_WIND"].notna() & train_df["AIR_TEMPERATURE"].notna() & train_df["BN"].notna()].index
train_not_null_df = train_df.loc[not_null_idx]

In [18]:
train_df.drop(columns=["U_WIND", "V_WIND", "AIR_TEMPERATURE", "BN"], inplace=True)

In [19]:
not_null_idx = test_df[test_df["U_WIND"].notna() & test_df["V_WIND"].notna() & test_df["AIR_TEMPERATURE"].notna() & test_df["BN"].notna()].index
test_not_null_df = test_df.loc[not_null_idx]

In [20]:
test_df.drop(columns=["U_WIND", "V_WIND", "AIR_TEMPERATURE", "BN"], inplace=True)

In [22]:
train_df.to_parquet("../../../train_v1_remove.parquet") # column 삭제
train_not_null_df.to_parquet("../../../train_v1_remain.parquet") # column 유지
test_df.to_parquet("../../../test_v1_remove.parquet")
test_not_null_df.to_parquet("../../../test_v1_remain.parquet")